In [1]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc

In [2]:
# Read the Excel files
df = pd.read_excel("staff.xlsx")
df_types = pd.read_excel("types.xlsx")
df_departments = pd.read_excel("departments.xlsx")

# Combine dataframes
df = df.merge(df_departments, on='Job', how='left')

# Create dropdown menu options
project_options = [{'label': project, 'value': project} for project in df['Project'].unique()]
type_options = [{'label': project_type, 'value': project_type} for project_type in df_types['Type'].unique()]
department_options = [{'label': 'Select All', 'value': 'all'}] + [
    {'label': department, 'value': department} for department in df_departments['Department'].unique()
]

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [3]:
# Define the layout of the app
app.layout = html.Div([
    html.H1("Deployment Plan"),
    dbc.Button("New Project", id="open", color="primary", className="mr-1"),
    dbc.Modal(
        [
            dbc.ModalHeader("New Project Details"),
            dbc.ModalBody(
                [
                    dcc.Input(id='type-input', type='text', placeholder='Type of Project'),
                    dcc.Input(id='duration-input', type='text', placeholder='Duration of Project')
                ]
            ),
            dbc.ModalFooter(
                [
                    dbc.Button("Close", id="close", className="ml-auto"),
                    dbc.Button("Save Changes", id="save-changes", className="ml-1")
                ]
            ),
        ],
        id="modal",
        size='lg',
        centered=True,
        backdrop='static'
    ),
    dcc.Dropdown(id='project-dropdown', options=project_options, value=project_options[0]['value']),
    dcc.Dropdown(id='department-dropdown', options=department_options, value='all'),
    dcc.Html(id='deployment-plan'),  # Use dcc.Html for the table
    html.Div(id='selected-project-type')  # Show the selected project type
])

AttributeError: module 'dash.dcc' has no attribute 'Html'

In [ ]:
# Callback to update the deployment plan (table) based on selected project and department
@app.callback(
    Output('deployment-plan', 'children'),
    [Input('project-dropdown', 'value'), Input('department-dropdown', 'value')]
)
def update_plan(selected_project, selected_department):
    if selected_project is None:
        return []  # Handle initial case

    # Filter data based on project and department
    filtered_data = df[
        (df['Project'] == selected_project) & (df['Department'] == selected_department)  # Filter by both
    ] if selected_department != 'all' else df[df['Project'] == selected_project]  # Filter only by project

    if filtered_data.empty:
        return []  # Handle empty data

    # Create a pivot table to summarize staff count by job and month
    staff_table = filtered_data.pivot_table(
        index='Job', columns='Month', values='Job', aggfunc='count'
    ).fillna(0)  # Fill missing values with 0

    return staff_table.to_html(index=True, justify='center')  # Convert to HTML table

In [ ]:
# New callback for the modal popup
@app.callback(
    Output("modal", "is_open"),
    [Input("open", "n_clicks"), Input("close", "n_clicks"), Input("save-changes", "n_clicks")],
    [dash.dependencies.State("modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open


In [ ]:
# Callback to show the project type for the selected project
@app.callback(
    Output('selected-project-type', 'children'),
    [Input('project-dropdown', 'value')]
)
def display_project_type(selected_project):
    # Get the type of the selected project
    project_type = df_types[df_types['Project'] == selected_project]['Type'].values[0]
    return f'Type of Project: {project_type}'


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)